In [7]:
import sqlalchemy as sa
from sqlalchemy.dialects.postgresql import VARCHAR, NUMERIC
import pandas as pd

In [8]:
# Criação do DataFrame
path = r'C:\Users\Fernando\Documents\CEPID - Projeto ARIES\data\brutos\SIH\csv\concat\concat_RDSP02_filtered.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')

df.head()

,ANO_CMPT,MES_CMPT,DT_INTER,DT_SAIDA,CEP,MUNIC_RES,MUNIC_MOV,CGC_HOSP,CNES,NASC,...,DIAGSEC2,DIAGSEC3,DIAGSEC4,DIAGSEC5,DIAGSEC6,DIAGSEC7,DIAGSEC8,DIAGSEC9,CID_MORTE,CID_ASSO
0,2014,2,20140130,20140131,11025011,354850,354850,,2042894,19510427,...,,,,,,,,,,
1,2014,2,20140105,20140107,5305000,355030,355030,63025530008512,2076926,19920220,...,,,,,,,,,,
2,2014,2,20140128,20140131,5735070,355030,355030,46392148003054,2786680,19990528,...,,,,,,,,,,
3,2014,2,20140205,20140205,14093150,354340,354340,57722118000140,2082187,20070813,...,,,,,,,,,,
4,2014,2,20140122,20140205,12910000,350760,350760,53221255005100,2704900,20140122,...,,,,,,,,,,


In [9]:
class tabelaCheia:
    def __init__(self, namebd):
        self.nomeTabela = namebd

        # Definição dos tipos de dados e tamanhos
        self.colunas = {
            'ANO_CMPT': VARCHAR(4),
            'MES_CMPT': VARCHAR(2),
            'DT_INTER': VARCHAR(8),
            'DT_SAIDA': VARCHAR(8),
            'CEP': VARCHAR(8),
            'MUNIC_RES': VARCHAR(6),
            'MUNIC_MOV': VARCHAR(6),
            'CGC_HOSP': VARCHAR(16),
            'CNES': VARCHAR(7),
            'NASC': VARCHAR(8),
            'SEXO': VARCHAR(1),
            'IDADE': NUMERIC(2),
            'COD_IDADE': VARCHAR(1),
            'NACIONAL': VARCHAR(3),
            'INSTRU': VARCHAR(1),
            'RACA_COR': VARCHAR(2),
            'ETNIA': VARCHAR(4),
            'CBOR': VARCHAR(6),
            'MORTE': NUMERIC(1),
            'UTI_MES_TO': NUMERIC(3),
            'MARCA_UTI': VARCHAR(2),
            'VAL_UTI': NUMERIC(9, 2),
            'PROC_SOLIC': VARCHAR(10),
            'PROC_REA': VARCHAR(10),
            'VAL_SH': NUMERIC(13, 2),
            'VAL_SP': NUMERIC(13, 2),
            'N_AIH': VARCHAR(13),
            'VAL_TOT': NUMERIC(14, 2),
            'INFEHOSP': VARCHAR(1),
            'DIAG_PRINC': VARCHAR(4),
            'DIAG_SECUN': VARCHAR(4),
            'DIAGSEC1': VARCHAR(4),
            'DIAGSEC2': VARCHAR(4),
            'DIAGSEC3': VARCHAR(4),
            'DIAGSEC4': VARCHAR(4),
            'DIAGSEC5': VARCHAR(4),
            'DIAGSEC6': VARCHAR(4),
            'DIAGSEC7': VARCHAR(4),
            'DIAGSEC8': VARCHAR(4),
            'DIAGSEC9': VARCHAR(4),
            'CID_MORTE': VARCHAR(4),
            'CID_ASSO': VARCHAR(4)
        }

    def __str__(self):
        return f'Tabela {self.nomeTabela}'

    def criaTabela(self, conn):
        metadata = sa.MetaData(bind=conn)
        metadata.reflect()

        if self.nomeTabela in metadata.tables:
            print(f"Tabela {self.nomeTabela} já existe.")
            return

        # Criar tabela com tipos e tamanhos específicos
        tabela = sa.Table(
            self.nomeTabela, metadata,
            *[sa.Column(coluna, tipo) for coluna, tipo in self.colunas.items()]
        )
        tabela.create(conn)
        print(f"Tabela {self.nomeTabela} criada com sucesso.")

    def insereDados(self, df, conn):
        try:
            df.to_sql(
                self.nomeTabela,
                con=conn,
                if_exists='append',
                index=False,
                dtype=self.colunas  # Garante que os tipos sejam preservados
            )
            print(f"Dados inseridos na tabela {self.nomeTabela}.")
        except Exception as e:
            print(f"Erro ao inserir dados na tabela {self.nomeTabela}: {e}")

    def removeTabela(self, conn):
        metadata = sa.MetaData(bind=conn)
        metadata.reflect()

        if self.nomeTabela not in metadata.tables:
            print(f"Tabela {self.nomeTabela} não existe.")
            return

        tabela = metadata.tables[self.nomeTabela]
        tabela.drop(conn)
        print(f"Tabela {self.nomeTabela} removida com sucesso.")


In [10]:
# Conexão com o banco de dados
nome_bd = 'teste'
usuario = 'postgres'
senha = 'dan313852*'
host = 'localhost'
porta = '5432'

conn = sa.create_engine(f'postgresql://{usuario}:{senha}@{host}:{porta}/{nome_bd}')

In [16]:
# Criando a tabela
tabela = tabelaCheia('sih')

Tabela testando criada.


In [17]:
# Inserindo os dados
tabela.insere_dados(df.head(10), conn)

Dados inseridos na tabela testando.


In [18]:
# Desconectando do banco
desconecting_database(conn)

Conexão fechada!
